In [2]:
import pandas as pd
import pickle
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np
import itertools
import pyarrow.parquet as pq
from tqdm import tqdm

In [5]:
df = pd.read_pickle('data_one_hot_sparse.pickle')
df

,100136,100508,100521,100682,101356,101396,101421,101750,101788,101844,...,104227,105003,103746,102256,105930,102021,104099,103200,100679,109058
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120719,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120720,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120721,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120722,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Подготовка тестового датасета

In [3]:
test_public_mfti = pq.read_table('test_public_mfti.parquet').to_pandas()

list(test_public_mfti.cookie_id)

train = pq.read_table('train_mfti.parquet').to_pandas()

mask=train.iloc[:, 3:4].stack().apply(lambda x: x in list(test_public_mfti.cookie_id)).unstack()

test_r=train[(mask.cookie_id)]

good_click_list=['click_contacts',
 'preview_click_contacts',
 'preview_click_phone',
 'preview_click_response',
 'click_phone',
 'click_response']
mask1=test_r.iloc[:, 5:6].stack().apply(lambda x: x in good_click_list).unstack()
test_r=test_r[(mask1.event_type)]


test_r=test_r[['vacancy_id_', 'cookie_id']]

test_res=test_r.groupby(['cookie_id', 'vacancy_id_']).agg({'vacancy_id_':'count'}).unstack().fillna (0)
test_res.columns = test_res.columns.droplevel(0)

In [13]:
test_public_mfti

,cookie_id,vacancy_id_
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812..."
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348..."
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065..."
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]"
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]"
...,...,...
767,fdbcda17f22f406486837059e76c7fed,"[207851, 254989, 213344, 214180, 222146]"
768,fe6193ab26494ace9be5aae36e507618,"[115352, 230546, 225527, 120188, 109360, 23212..."
769,fe95b2826ee1452b81201ed3f4c3294d,"[240362, 114852, 253946, 251081, 127546, 244688]"
770,ff1aef256a49481698bb2e938510ff36,"[231194, 236363, 220747, 244688, 100094, 24052..."


772-628=144 пользователя не сделали ни одного положительного действия



In [14]:
test_res.index[:4]

Index(['000cd76cd33f43d4a1ac1d16d10f8bf7', '0034bc7f404341ba8412665453e7825a',
       '00a6c5a64a274c55a836402bdeb3b2c4', '015937a125b14e74bdff1cddc49f9172'],
      dtype='object', name='cookie_id')

In [15]:
def sp_loc(df, index, columns, val):
    """ Insert data in a DataFrame with SparseDtype format

    Only applicable for pandas version > 0.25

    Args
    ----
    df : DataFrame with series formatted with pd.SparseDtype
    index: str, or list, or slice object
        Same as one would use as first argument of .loc[]
    columns: str, list, or slice
        Same one would normally use as second argument of .loc[]
    val: insert values

    Returns
    -------
    df: DataFrame
        Modified DataFrame

    """

    # Save the original sparse format for reuse later
    spdtypes = df.dtypes[columns]

    # Convert concerned Series to dense format
    df[columns] = df[columns].sparse.to_dense()

    # Do a normal insertion with .loc[]
    df.loc[index, columns] = val

    # Back to the original sparse format
    df[columns] = df[columns].astype(spdtypes)

    return df

Запуск алгоритма рекомендации

In [16]:
sum_metric=[]
for random_user in tqdm(test_res.index):
    try:
    #Находим пересечение вакансий нашего датасета и вакансий пользователя
    
        vacancy_watched =list(set(test_res.loc[random_user, :][test_res.loc[random_user, :]>0].index)& set(df.columns))
    

    #Найдем 50 наиболее похожих пользователей:

        df_random_user=df[vacancy_watched]


        df_top_50=df_random_user.T.sum()

    # df_top_50.sort_values(ascending=False)[0:51] - первые 50 пользователей включая и самого

        data_same_user=df_random_user.loc[list(df_top_50.sort_values(ascending=False)[0:30].index), :]

    # строим матрицу схожести добавив в конец самого user (если он есть у нас все равно мы будем занулять все вакансии )
    
        user_similarity = pairwise_distances(sp_loc(data_same_user, len(data_same_user), vacancy_watched, 1), metric='cosine').round(2)

    # Отбираем топ 10 похожих пользователей

        s=pd.Series(index=data_same_user.index[:-1], data=(1-user_similarity[-1][:-1]))
    
        s.sort_values(ascending=False)[0:10].index

    #Перемножаем на получившиеся коэф схожести ряды вакансий наших пользлвателей и складываем их
    
        kf=s.sort_values(ascending=False)[0:10]

        itogo=df.loc[kf.index, :]

        d=np. zeros(103237)

        for i in kf.index:
            d+=itogo.loc[i, :].to_numpy()*kf[i]
    
        last_sort=pd.Series(index=itogo.columns, data=d)

        # Зануляем вакансии которые у нас уже использовались

        last_sort[vacancy_watched]=0

        a=set(last_sort.sort_values(ascending=False)[:5].index)
        b=set(test_public_mfti.vacancy_id_[test_public_mfti.cookie_id==random_user].to_list()[0])
    
        sum_metric.append(len(a & b)/5)
    except:
        print(i)

 44%|████▍     | 279/628 [1:18:04<1:29:08, 15.32s/it]

56473


 52%|█████▏    | 325/628 [1:30:32<1:16:53, 15.23s/it]

76975


 63%|██████▎   | 394/628 [1:49:57<1:05:36, 16.82s/it]

...

 80%|████████  | 503/628 [2:19:12<39:34, 18.99s/it]

112818


 90%|█████████ | 567/628 [2:36:16<16:53, 16.61s/it]

29465


100%|██████████| 628/628 [2:53:26<00:00, 16.57s/it]


In [17]:
sum(sum_metric)/len(sum_metric)

0.04573268921095